In [113]:
import re
import pandas as pd

In [114]:
# pull the command from command.txt, if available
# pull the training scores from log.txt
# pull the evaluation scores from scores.txt

In [115]:
filepath = '/home/shane/minerl/logs/shane_logs/'

In [116]:
#s = "[('average_q', 0.02713955), ('average_loss', 0.06054627889068797), ('cumulative_steps', 22183), ('n_updates', 4293), ('rlen', 22183)]"

In [117]:
#eval(s)

In [118]:
#s2 = "[('average_q', nan), ('average_loss', nan), ('cumulative_steps', 2000), ('n_updates', 0), ('rlen', 2000)]"

In [119]:
# df = pd.DataFrame(eval(s2)) problems with 'nan'
# df = pd.read_csv(f"{filepath}log.txt")
# readcsv doesnt work as lines are irregular. eval has problems b/c doesn't recognize nans
#import ast
#ast.literal_eval(s2) malformed ... try something else?

In [120]:
# shell command attempt
#log_file = f"{filepath}log.txt"
#stat_file = f"{filepath}statistics.txt"
#reward_file = f"{filepath}rewards.txt"
#! grep 'statistics:' {log_file} > {stat_file}
#! grep 'outdir:result' {log_file} > {reward_file}

In [121]:
rx_dict = {
    'statistics': re.compile(r"statistics:(?P<statistics>.*)\n"),
    'result': re.compile(r"outdir:result/.* (?P<result>step:(\d+) episode:(\d+) R:(.*))\n")
}
def _parse_line(line):
    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            #print(key)
            return key, match
    return None, None

In [122]:
statistics = []
rewards = []

with open(f"{filepath}log.txt", 'r') as f:
    for line in f:
        key, match = _parse_line(line)
        if key == 'statistics':
            # replace nan with 0
            stats = match.group('statistics').replace('nan', '0.0')
            # convert the list of tuples into a dict
            statistics.append(dict(eval(stats)))
            
        if key == 'result':
            result = match.group('result').replace('nan', '0.0').split()
            rewards.append(dict(map(lambda s : s.split(':'), result)))
            # the dict values are strings, should be int. convert here, or in dataframe?
            #print(steps, episode, reward)
            
df = pd.DataFrame(rewards).apply(pd.to_numeric)
#df2 = pd.DataFrame(statistics).set_index('cumulative_steps')

#df.join(pd.DataFrame(statistics).set_index('cumulative_steps'), on='step')
#df.join(df2, on='step')

df = df.join(pd.DataFrame(statistics).set_index('cumulative_steps'), on='step')

In [123]:
df

,step,episode,R,average_q,average_loss,n_updates,rlen
0,2000,0,0.0,0.000000,0.000000,0,2000
1,4000,1,1.0,0.000000,0.000000,0,4000
2,6000,2,0.0,0.103058,0.141552,248,6000
3,8000,3,0.0,0.153867,0.112361,748,8000
4,10000,4,0.0,0.107413,0.104383,1248,10000
...,...,...,...,...,...,...,...
856,1672621,856,47.0,2.917360,0.058864,416903,300000
857,1674621,857,55.0,2.839762,0.055643,417403,300000
858,1676621,858,50.0,2.947097,0.053313,417903,300000
859,1678621,859,53.0,2.896661,0.056821,418403,300000
